In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('oxc20-21data.txt')
winter2 = pd.read_csv ('oxc21-22data.txt')
winter3 = pd.read_csv ('oxc22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [7]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 51]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 51]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 51]

In [8]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
oxc = LocationInfo(
    'Oxford', 'Connecticut', 
    latitude=41.4338,
    longitude=- 73.1168,
    timezone='America/Connecticut'
)

oxc

LocationInfo(name='Oxford', region='Connecticut', timezone='America/Connecticut', latitude=41.4338, longitude=-73.1168)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(oxc.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(oxc.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(oxc.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:00:01  2020-12-01 16:23:13  2020-12-01
1   2020-12-02 07:01:02  2020-12-02 16:22:59  2020-12-02
2   2020-12-03 07:02:03  2020-12-03 16:22:48  2020-12-03
3   2020-12-04 07:03:02  2020-12-04 16:22:38  2020-12-04
4   2020-12-05 07:03:59  2020-12-05 16:22:31  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:34:30  2021-02-24 17:37:18  2021-02-24
86  2021-02-25 06:32:59  2021-02-25 17:38:30  2021-02-25
87  2021-02-26 06:31:28  2021-02-26 17:39:41  2021-02-26
88  2021-02-27 06:29:56  2021-02-27 17:40:52  2021-02-27
89  2021-02-28 06:28:23  2021-02-28 17:42:02  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:59:46  2021-12-01 16:23:17  2021-12-01
1   2021-12-02 07:00:48  2021-12-02 16:23:02  2021-12-02
2   2021-12-03 07:01:48  2021-12-03 16:22:50  2021-12-03
3   2021-12-04 07:02:47  2021-12-04 16:22:40  2021-12-04
4   2021-12-05 07:03:46  2021-12-05 16:22:32  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:34:51  2022-02-24 17:37:01  2022-02-24
86  2022-02-25 06:33:21  2022-02-25 17:38:12  2022-02-25
87  2022-02-26 06:31:50  2022-02-26 17:39:24  2022-02-26
88  2022-02-27 06:30:18  2022-02-27 17:40:35  2022-02-27
89  2022-02-28 06:28:45  2022-02-28 17:41:45  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:59:31  2022-12-01 16:23:20  2022-12-01
1   2022-12-02 07:00:33  2022-12-02 16:23:05  2022-12-02
2   2022-12-03 07:01:34  2022-12-03 16:22:52  2022-12-03
3   2022-12-04 07:02:33  2022-12-04 16:22:42  2022-12-04
4   2022-12-05 07:03:32  2022-12-05 16:22:34  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:35:13  2023-02-24 17:36:43  2023-02-24
86  2023-02-25 06:33:43  2023-02-25 17:37:55  2023-02-25
87  2023-02-26 06:32:12  2023-02-26 17:39:06  2023-02-26
88  2023-02-27 06:30:40  2023-02-27 17:40:17  2023-02-27
89  2023-02-28 06:29:08  2023-02-28 17:41:28  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,OXC,2020-12-05 14:51:00,3.5600,-SN BR,2020-12-05,2020-12-05 07:03:59,2020-12-05 16:22:31
1,OXC,2020-12-05 15:51:00,1.7800,-SN BR,2020-12-05,2020-12-05 07:03:59,2020-12-05 16:22:31
2,OXC,2020-12-05 16:51:00,1.2700,-SN BR,2020-12-05,2020-12-05 07:03:59,2020-12-05 16:22:31
3,OXC,2020-12-05 17:51:00,1.0200,-SN BR,2020-12-05,2020-12-05 07:03:59,2020-12-05 16:22:31
4,OXC,2020-12-09 19:51:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:07:37,2020-12-09 16:22:27
...,...,...,...,...,...,...,...
190,OXC,2021-02-22 18:51:00,1.2700,-SN FG,2021-02-22,2021-02-22 06:37:28,2021-02-22 17:34:54
191,OXC,2021-02-22 19:51:00,0.7600,-SN FG,2021-02-22,2021-02-22 06:37:28,2021-02-22 17:34:54
192,OXC,2021-02-27 08:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:29:56,2021-02-27 17:40:52
193,OXC,2021-02-27 09:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:29:56,2021-02-27 17:40:52


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,OXC,2021-12-08 19:51:00,0.0001,-SN BR,2021-12-08,2021-12-08 07:06:32,2021-12-08 16:22:24
1,OXC,2021-12-08 20:51:00,0.0001,-SN BR,2021-12-08,2021-12-08 07:06:32,2021-12-08 16:22:24
2,OXC,2021-12-08 21:51:00,0.0001,-SN BR,2021-12-08,2021-12-08 07:06:32,2021-12-08 16:22:24
3,OXC,2021-12-08 22:51:00,0.0001,-SN BR,2021-12-08,2021-12-08 07:06:32,2021-12-08 16:22:24
4,OXC,2021-12-08 23:51:00,0.0001,-SN BR,2021-12-08,2021-12-08 07:06:32,2021-12-08 16:22:24
...,...,...,...,...,...,...,...
124,OXC,2022-02-25 13:51:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:33:21,2022-02-25 17:38:12
125,OXC,2022-02-25 14:51:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:33:21,2022-02-25 17:38:12
126,OXC,2022-02-25 15:51:00,0.0001,-SN,2022-02-25,2022-02-25 06:33:21,2022-02-25 17:38:12
127,OXC,2022-02-25 18:51:00,4.5700,-SN BR,2022-02-25,2022-02-25 06:33:21,2022-02-25 17:38:12


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,OXC,2022-12-11 13:51:00,0.0001,-SN,2022-12-11,2022-12-11 07:08:53,2022-12-11 16:22:34
1,OXC,2022-12-11 16:51:00,0.0001,-SN,2022-12-11,2022-12-11 07:08:53,2022-12-11 16:22:34
2,OXC,2022-12-11 17:51:00,0.0001,-SN BR,2022-12-11,2022-12-11 07:08:53,2022-12-11 16:22:34
3,OXC,2022-12-11 18:51:00,0.0001,-SN,2022-12-11,2022-12-11 07:08:53,2022-12-11 16:22:34
4,OXC,2022-12-11 19:51:00,0.0001,-SN BR,2022-12-11,2022-12-11 07:08:53,2022-12-11 16:22:34
5,OXC,2022-12-11 20:51:00,0.0001,-SN BR,2022-12-11,2022-12-11 07:08:53,2022-12-11 16:22:34
6,OXC,2022-12-11 21:51:00,0.0001,-SN BR,2022-12-11,2022-12-11 07:08:53,2022-12-11 16:22:34
7,OXC,2022-12-11 22:51:00,0.0001,-SN BR,2022-12-11,2022-12-11 07:08:53,2022-12-11 16:22:34
8,OXC,2022-12-11 23:51:00,0.0001,-SN BR,2022-12-11,2022-12-11 07:08:53,2022-12-11 16:22:34
9,OXC,2022-12-12 00:51:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:09:41,2022-12-12 16:22:43


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,OXC,2020-12-05 14:51:00,3.5600,-SN BR,2020-12-05,2020-12-05 07:03:59,2020-12-05 16:22:31
1,OXC,2020-12-05 15:51:00,1.7800,-SN BR,2020-12-05,2020-12-05 07:03:59,2020-12-05 16:22:31
19,OXC,2020-12-17 07:51:00,0.2500,-SN BR,2020-12-17,2020-12-17 07:13:35,2020-12-17 16:24:10
20,OXC,2020-12-17 08:51:00,0.0001,-SN BR,2020-12-17,2020-12-17 07:13:35,2020-12-17 16:24:10
21,OXC,2020-12-17 09:51:00,0.0001,-SN BR,2020-12-17,2020-12-17 07:13:35,2020-12-17 16:24:10
...,...,...,...,...,...,...,...
187,OXC,2021-02-20 14:51:00,0.0001,-SN,2021-02-20,2021-02-20 06:40:23,2021-02-20 17:32:29
188,OXC,2021-02-22 12:51:00,0.0001,-SN,2021-02-22,2021-02-22 06:37:28,2021-02-22 17:34:54
192,OXC,2021-02-27 08:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:29:56,2021-02-27 17:40:52
193,OXC,2021-02-27 09:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:29:56,2021-02-27 17:40:52


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
0,OXC,2020-12-05 14:51:00,3.5600,-SN BR,2020-12-05,2020-12-05 07:03:59,2020-12-05 16:22:31,2020-12-05 08:03:59,2020-12-05 15:22:31
20,OXC,2020-12-17 08:51:00,0.0001,-SN BR,2020-12-17,2020-12-17 07:13:35,2020-12-17 16:24:10,2020-12-17 08:13:35,2020-12-17 15:24:10
21,OXC,2020-12-17 09:51:00,0.0001,-SN BR,2020-12-17,2020-12-17 07:13:35,2020-12-17 16:24:10,2020-12-17 08:13:35,2020-12-17 15:24:10
22,OXC,2020-12-17 10:51:00,0.2500,+SN BLSN FZFG,2020-12-17,2020-12-17 07:13:35,2020-12-17 16:24:10,2020-12-17 08:13:35,2020-12-17 15:24:10
23,OXC,2020-12-17 11:51:00,0.0001,-SN BLSN BR,2020-12-17,2020-12-17 07:13:35,2020-12-17 16:24:10,2020-12-17 08:13:35,2020-12-17 15:24:10
...,...,...,...,...,...,...,...,...,...
187,OXC,2021-02-20 14:51:00,0.0001,-SN,2021-02-20,2021-02-20 06:40:23,2021-02-20 17:32:29,2021-02-20 07:40:23,2021-02-20 16:32:29
188,OXC,2021-02-22 12:51:00,0.0001,-SN,2021-02-22,2021-02-22 06:37:28,2021-02-22 17:34:54,2021-02-22 07:37:28,2021-02-22 16:34:54
192,OXC,2021-02-27 08:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:29:56,2021-02-27 17:40:52,2021-02-27 07:29:56,2021-02-27 16:40:52
193,OXC,2021-02-27 09:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:29:56,2021-02-27 17:40:52,2021-02-27 07:29:56,2021-02-27 16:40:52


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.1728391304347826
2021-2022: 0.05253584905660379
2022-2023: 0.05619444444444445


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
2,OXC,2020-12-05 16:51:00,1.2700,-SN BR,2020-12-05,2020-12-05 07:03:59,2020-12-05 16:22:31
3,OXC,2020-12-05 17:51:00,1.0200,-SN BR,2020-12-05,2020-12-05 07:03:59,2020-12-05 16:22:31
4,OXC,2020-12-09 19:51:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:07:37,2020-12-09 16:22:27
5,OXC,2020-12-14 16:51:00,0.5100,-SN BR,2020-12-14,2020-12-14 07:11:34,2020-12-14 16:23:14
6,OXC,2020-12-14 17:51:00,0.5100,-SN BR,2020-12-14,2020-12-14 07:11:34,2020-12-14 16:23:14
...,...,...,...,...,...,...,...
184,OXC,2021-02-19 23:51:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:41:48,2021-02-19 17:31:16
185,OXC,2021-02-20 00:51:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:40:23,2021-02-20 17:32:29
189,OXC,2021-02-22 17:51:00,0.0001,-SN FG,2021-02-22,2021-02-22 06:37:28,2021-02-22 17:34:54
190,OXC,2021-02-22 18:51:00,1.2700,-SN FG,2021-02-22,2021-02-22 06:37:28,2021-02-22 17:34:54


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4416\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.1763125
2021-2022: 0.2349
2022-2023: 0.15257


In [65]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'CT'

data.to_csv('SO.oxc.csv', index=False)